In [15]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pickle as pk
import resources.RES as res
from sklearn.preprocessing import minmax_scale as sk_minmax

In [16]:
# retrieve context, maximum extents, and maximum extends as voxelised volume

context_path = os.path.relpath("../data/movedcontext.obj") 
context_mesh = tm.load(context_path)

# extents_path = os.path.relpath("../data/maximumextents.obj") 
# extents_mesh = tm.load(extents_path)

env_lat_path = os.path.relpath("../data/macrovoxels.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [17]:
# skydome
skydome = pk.load(open("../data/skydome.pk", "rb"))

In [18]:
# create trimesh cuboids for computing intersections
def transform_mat(value):
    mat = np.identity(4)
    mat[:3,-1] = np.array(value)
    return mat

In [19]:
# voxel cuboid meshes
vox_cuboids = [tm.creation.box(envelope_lattice.unit, transform=transform_mat(ct)) for ct in envelope_lattice.centroids]

# number of faces per voxel
f_numb = len(vox_cuboids[0].faces)

# combine voxels into one mesh
combined_voxels = tm.util.concatenate(vox_cuboids)

In [20]:
# combine voxels and context into one mesh
combined_meshes = tm.util.concatenate(combined_voxels, context_mesh)

# shoot towards the skydome points from all of the voxels
DL_ray_ctr = np.tile(skydome, [len(envelope_lattice.centroids),1]) # daylighting ray for each centroid
DL_ctr_ray = np.tile(envelope_lattice.centroids, [1, len(skydome)]).reshape(-1, 3) # daylighting centroid for each ray

In [21]:
# intersection of rays from voxel centroids to sky patch objects with all voxel faces
face_id, ray_id = combined_meshes.ray.intersects_id(ray_origins=DL_ctr_ray, ray_directions=DL_ray_ctr, multiple_hits=True) # environment too complex? Takes about 70s

In [22]:
G1, U1 = res.construct_graph(skydome, face_id, ray_id, envelope_lattice, f_numb)

In [23]:
# save interedependencies for later use
pk.dump(G1, open("../data/SkyG1.pk", "wb"))

# save obstructions for later use
pk.dump(U1, open("../data/SkyU1.pk", "wb"))

In [24]:
# sum per row twice
sky_obstructed = (G1.sum(axis=1)).sum(axis=1)

# sum per column and then per row
sky_obstructing = (G1.sum(axis=0)).sum(axis=1)

In [25]:
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [26]:
# apply weight to sky patches/points (currently weight of 1 uniformly)
w1 = np.ones(len(skydome))

# dot product with ray weights
o1 = np.array(np.dot(G1.sum(axis=1),w1), dtype='int64') # sun blocking
o2 = np.array(np.dot(G1.sum(axis=0),w1), dtype='int64') # sun blocked

# obscurity cost formula (hadamard product)
c2 = np.multiply(o1+1, o2+1)

# normalize values
c2_norm = sk_minmax(c2) # voxel daylighting potential

# reshape and store to lattice
c2_lat = reshape_and_store_to_lattice(c2_norm, envelope_lattice)

In [27]:
# save normalized values for later use
pk.dump(c2_norm, open("../data/c2_norm.pk", "wb"))
# normalized values of sky blocking/sky blocked cost of each voxel

In [28]:
# visualize daylighting potential values 
p = pv.Plotter(notebook=True)

def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# fast visualization of the lattice
# envelope_lattice.fast_vis(p)

# add context
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

base_lattice = c2_lat
grid = pv.UniformGrid() # Create the spatial reference
grid.dimensions = np.array(base_lattice.shape) + 1 # Set the grid dimensions
grid.origin = base_lattice.minbound - 0.5 * base_lattice.unit # The bottom left corner of the data set
grid.spacing = base_lattice.unit # These are the cell sizes along each axis

# Add the data values to the cell data
# grid.point_arrays["Score"] = base_lattice.flatten(order="F")  # Flatten the Lattice
grid.cell_arrays["Score"] = base_lattice.flatten(order="F")  # Flatten the Lattice
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.0
p.add_volume(grid, cmap="coolwarm", clim=[base_lattice.min(), base_lattice.max()],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(788.6785661399646, 711.2621611399645, 723.5957336399646),
 (65.08283250000001, -12.333572500000002, 0.0),
 (0.0, 0.0, 1.0)]